# Overview

These notes are a continouation of the previous set of notes. We will go through some more advanced usage of SQLAlchemy, including mass insertion, and advanced query features. We will then look under the hood for some details on how SQLAlchemy's features are implemented, using interesting programming and embedding techniques.

# Music Library Example

Our running example will be a database made out of Artists and Albums. We define the schema below.

In [24]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import validates, relationship

import math

engine = create_engine('sqlite:///:memory:', echo=True)
Base = declarative_base()

class Artist(Base):
    __tablename__ = 'artists' # the actual name in the DB

    id = Column(Integer, primary_key=True)
    name = Column(String, unique=True, nullable=False)
    year = Column(Integer)

    @validates('year')
    def year_is_valid(self, key, year):
        year = math.floor(year)
        if year < 1950 or year > 2019:
            raise ValueError('Illegal year!')
        return year
    
    def __repr__(self): # python side
        return "<Artist(id={}, name='{}', year={}>".format(self.id, self.name, self.year)

class Album(Base):
    __tablename__ = 'albums'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    tracks = Column(Integer, nullable=False)

    artist_id = Column(Integer, ForeignKey(Artist.id), nullable=False)
    artist = relationship('Artist', backref='albums', lazy='joined')

    def __repr__(self):
        return "<Album(id={}, name={}, #tracks={}, artist={})".format(
            self.id, self.name, self.tracks, self.artist.name)

Base.metadata.create_all(engine)

2019-11-14 08:48:52,665 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-11-14 08:48:52,666 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 08:48:52,667 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-11-14 08:48:52,667 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 08:48:52,668 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("albums")
2019-11-14 08:48:52,668 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 08:48:52,668 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("albums")
2019-11-14 08:48:52,669 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 08:48:52,669 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("artists")
2019-11-14 08:48:52,670 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 08:48:52,670 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("artists")
2019-11-14 08:48:52,671 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 08:48:52,

## Inserting Data

Now, we will insert data into our database. A common technique is to read data from a csv or JSON file and insert it into DB. Sometimes, or to scrape data online or acquire it from other databases. Here, we will start with some JSON dump of the data.

In [2]:
json_str = '''[
    {"type": "artist", "content": {"name": "Lynyrd Skynyrd", "year": 1964}},
    {"type": "artist", "content": {"name": "Led Zeppelin", "year": 1968}},
    {"type": "artist", "content": {"name": "Metallica", "year": 1981}},
    {"type": "artist", "content": {"name": "Slayer", "year": 1981}},
    {"type": "artist", "content": {"name": "Pantera", "year": 1981}},
    {"type": "artist", "content": {"name": "Pearl Jam"}},
    {"type": "artist", "content": {"name": "Tool"}},
    {"type": "album", "content": {"name": "Master of Puppets", "artist": "Metallica", "tracks": 8}},
    {"type": "album", "content": {"name": "...And Justice for All", "artist": "Metallica", "tracks": 9}},
    {"type": "album", "content": {"name": "Fear Inoculumn", "artist": "Tool", "tracks": 10}},
    {"type": "album", "content": {"name": "Seasons in the Abyss", "artist": "Slayer", "tracks": 10}},
    {"type": "album", "content": {"name": "Hell Awaits", "artist": "Slayer", "tracks": 7}},
    {"type": "album", "content": {"name": "Far Beyond Driven", "artist": "Pantera"}}
]'''

Our JSON data contains many artists and bands. However, our data is not clean. Some artists are missing their year of formation, some albums may be missing the number of tracks. If an artist fails to be inserted, we want all its albums not to be inserted. However, if an album fails, this should not affect other albums or the artist.

We can achieve this via committing every entry independently, starting with bands first. More complicated conditions can be implemented using subtransactions.

In [3]:
import json

from sqlalchemy.orm import sessionmaker
SessionMaker = sessionmaker(bind=engine)

session = SessionMaker()

data = json.loads(json_str)
artists = {}
for entry in data:
    if entry['type'] == 'artist':
        artist = Artist(**entry['content'])
        artists[entry['content']['name']] = artist
        session.add(artist)
        try:
            session.commit()
        except Exception as e:
            print('\n', e, '\n')
            
            del artists[entry['content']['name']]
            session.rollback()

for entry in data:
    if entry['type'] == 'album':
        artist = entry['content']['artist']
        del entry['content']['artist']
        
        album = Album(**entry['content'])
        if artist in artists:
            album.artist = artists[artist]
            session.add(album)
            try:
                session.commit()
            except Exception as e:
                print('\n', e, '\n')
                session.rollback

session = SessionMaker()
print('\n', session.query(Artist).all(), '\n')
print(session.query(Album).all())

2019-11-14 08:36:17,374 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 08:36:17,378 INFO sqlalchemy.engine.base.Engine INSERT INTO artists (name, year) VALUES (?, ?)
2019-11-14 08:36:17,379 INFO sqlalchemy.engine.base.Engine ('Lynyrd Skynyrd', 1964)
2019-11-14 08:36:17,384 INFO sqlalchemy.engine.base.Engine COMMIT
2019-11-14 08:36:17,387 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 08:36:17,390 INFO sqlalchemy.engine.base.Engine INSERT INTO artists (name, year) VALUES (?, ?)
2019-11-14 08:36:17,392 INFO sqlalchemy.engine.base.Engine ('Led Zeppelin', 1968)
2019-11-14 08:36:17,395 INFO sqlalchemy.engine.base.Engine COMMIT
2019-11-14 08:36:17,400 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 08:36:17,401 INFO sqlalchemy.engine.base.Engine INSERT INTO artists (name, year) VALUES (?, ?)
2019-11-14 08:36:17,402 INFO sqlalchemy.engine.base.Engine ('Metallica', 1981)
2019-11-14 08:36:17,405 INFO sqlalchemy.engine.base.Engine COMMIT
2019-11-14 

## Advanced Queries

We will make several queries that attempt to cover a variety of useful SQL features, including group by, aggregation, ordering, and limits.

### All Artists with Albums

In [4]:
from sqlalchemy import func

session = SessionMaker()

artists = session.query(Artist).filter(Artist.albums.any()).all()
for artist in artists:
    print('\n', artist, artist.albums, '\n')

2019-11-14 08:36:17,475 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 08:36:17,476 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists 
WHERE EXISTS (SELECT 1 
FROM albums 
WHERE artists.id = albums.artist_id)
2019-11-14 08:36:17,476 INFO sqlalchemy.engine.base.Engine ()
2019-11-14 08:36:17,478 INFO sqlalchemy.engine.base.Engine SELECT albums.id AS albums_id, albums.name AS albums_name, albums.tracks AS albums_tracks, albums.artist_id AS albums_artist_id 
FROM albums 
WHERE ? = albums.artist_id
2019-11-14 08:36:17,478 INFO sqlalchemy.engine.base.Engine (3,)

 <Artist(id=3, name='Metallica', year=> [<Album(id=1, name=Master of Puppets, #tracks=8, artist=Metallica), <Album(id=2, name=...And Justice for All, #tracks=9, artist=Metallica)] 

2019-11-14 08:36:17,480 INFO sqlalchemy.engine.base.Engine SELECT albums.id AS albums_id, albums.name AS albums_name, albums.tracks AS albums_track

### Artists With At Least Two Albums

In [34]:
from sqlalchemy import func, text

session = SessionMaker()

# This is not recommended, because it uses SQL as a string
print('query 1')
query = session.query(Artist, func.count(Album.id).label('count')).join(Album).group_by(Artist.id)
query = query.having(text('count > 1'))
artists = query.all()
for (artist, count) in artists:
    print(artist, count)

# A bit better
print('\nquery 2')
count = func.count(Album.id).label('count')
query = session.query(Artist, count).join(Album).group_by(Artist.id).having(count > 1)
for (artist, count) in query.all():
    print(artist, count)

query 1
2019-11-14 08:51:56,998 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 08:51:57,002 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year, count(albums.id) AS count 
FROM artists JOIN albums ON artists.id = albums.artist_id GROUP BY artists.id 
HAVING count > 1
2019-11-14 08:51:57,004 INFO sqlalchemy.engine.base.Engine ()
<Artist(id=3, name='Metallica', year=1981> 2
<Artist(id=4, name='Slayer', year=1981> 2

query 2
2019-11-14 08:51:57,011 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year, count(albums.id) AS count 
FROM artists JOIN albums ON artists.id = albums.artist_id GROUP BY artists.id 
HAVING count(albums.id) > ?
2019-11-14 08:51:57,012 INFO sqlalchemy.engine.base.Engine (1,)
<Artist(id=3, name='Metallica', year=1981> 2
<Artist(id=4, name='Slayer', year=1981> 2


### Get The Two Bands With The Most Number of Tracks

In [36]:
session = SessionMaker()

tracks = func.sum(Album.tracks).label('tracks')
query = session.query(Artist, tracks).join(Album)
query = query.group_by(Artist.id)
query = query.order_by(tracks.desc())
query = query.limit(2)
for (artist, tracks) in query.all():
    print(artist, tracks)

2019-11-14 08:54:31,257 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-11-14 08:54:31,261 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year, sum(albums.tracks) AS tracks 
FROM artists JOIN albums ON artists.id = albums.artist_id GROUP BY artists.id ORDER BY tracks DESC
 LIMIT ? OFFSET ?
2019-11-14 08:54:31,262 INFO sqlalchemy.engine.base.Engine (2, 0)
<Artist(id=3, name='Metallica', year=1981> 17
<Artist(id=4, name='Slayer', year=1981> 17


# Behind The Scenes

Here we will look more at how SQLAlchemy implements certain features behind the scenes.

## Declarative Base

How does SQLAlchemy keep track of which python classes are modules? Modules inherit from a declarative base. However, inheritance is a passive operation. It does not notify the parent class of the act of inheritance (not at definition time at least).

The easy solution for this would be to require developers to manually register their modules, by calling some function. However, python provide a better automated solution via MetaClasses: [source](https://github.com/sqlalchemy/sqlalchemy/tree/master/lib/sqlalchemy/ext).

## Meta Classes

A Meta class, is a class whose instances are classes themselves. It is a class that gives classes instead of objects.

A meta class can be written in python by inheriting from type.

In [56]:
class RegisteringMetaClass(type):
    def __init__(cls, clsname, bases, attrs):
        print('register', clsname, bases, attrs.keys())
        return super().__init__(clsname, bases, attrs)

class MyClass(object, metaclass=RegisteringMetaClass):
    attr = 'my attribute'
    def func():
        pass

class MyClass2(MyClass):
    attr2 = 'other attribute'

class MyClass3():
    pass

register MyClass (<class 'object'>,) dict_keys(['func', '__qualname__', 'attr', '__module__'])
register MyClass2 (<class '__main__.MyClass'>,) dict_keys(['__qualname__', '__module__', 'attr2'])


The above example uses the metaclass=<metaclass> syntax, which is explicit but not regular. SQLAlchemy classes inherit from Base directly without refering to MetaClass. This can be achieved by ensuring that Base is a class defined with metaclass=<Metaclass>, or by getting an instance of the metaclass (whose instances are classes themselves), and using it as the parent class in inheritance.

In [71]:
def mygetattr(self, key, value):
    print('instrumented', key, value)
    super(self.__class__, self).__setattr__(key, value)

class InstrumentingMetaClass(type):
    def __new__(cls, clsname, bases, attrs):
        if clsname != 'Base':
            print('instrument', clsname, '\n')
            attrs['__setattr__'] = mygetattr
        return super().__new__(cls, clsname, bases, attrs)

MetaClassInstance = InstrumentingMetaClass('Base', (object,), {})
print('MetaClassInstance =', MetaClassInstance)

class ConcreteClass(MetaClassInstance):
    def __init__(self):
        self.my_attr = 'Test'

instance = ConcreteClass()
instance.my_attr = 'Test 2'

MetaClassInstance = <class '__main__.Base'>
instrument ConcreteClass 

instrumented my_attr Test
instrumented my_attr Test 2


## Query Implementation

Sources: [Query](https://github.com/zzzeek/sqlalchemy/blob/master/lib/sqlalchemy/orm/query.py).

In [81]:
from sqlalchemy import and_

query = session.query(Artist)
print('type(query) =', type(query))
print('query = ', query)

condition1 = Artist.year > 1980
condition2 = Artist.year <= 1990
condition3 = and_(condition1, condition2)

print('')
print(condition1)
print(type(condition1))

print('')
print(condition2)
print(type(condition2))

print('')
print(condition3)
print(type(condition3))
print(and_)

print('')
query = query.filter(condition3)
print(query)

print('')
print(query.all())

type(query) = <class 'sqlalchemy.orm.query.Query'>
query =  SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists

artists.year > :year_1
<class 'sqlalchemy.sql.elements.BinaryExpression'>

artists.year <= :year_1
<class 'sqlalchemy.sql.elements.BinaryExpression'>

artists.year > :year_1 AND artists.year <= :year_2
<class 'sqlalchemy.sql.elements.BooleanClauseList'>
<function and_ at 0x7f34e41b8598>

SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists 
WHERE artists.year > ? AND artists.year <= ?

2019-11-14 09:19:35,506 INFO sqlalchemy.engine.base.Engine SELECT artists.id AS artists_id, artists.name AS artists_name, artists.year AS artists_year 
FROM artists 
WHERE artists.year > ? AND artists.year <= ?
2019-11-14 09:19:35,508 INFO sqlalchemy.engine.base.Engine (1980, 1990)
[<Artist(id=3, name='Metallica', year=1981>, <Artist(id=4, name='Slayer', year=1981>, <Artist(id=5, name='Pa